In [1]:
old_context = """
Summarize the following dialog focusing only on the key information shared by the user.
[Dialog]
{dialog}
"""

old_functions = [
    {
        "name": "summarize_conversation",
        "description": "Creates a focused summary of the user's shared information from the conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": """
[Task]
Create a concise summary in Korean focusing only on the key information shared by the user.

[Rules]
1. Focus only on the actual information and preferences shared by the user
2. Exclude meta-information about the conversation structure
3. Don't mention that this was a conversation or dialogue
4. Omit references to the assistant/AI/비서
5. Write in a straightforward, factual style

[Example Format]
기분이 좋다고 했으며, 오늘 딸기가 들어간 크림 모카번을 먹었다. 커피는 잠을 못 자는 문제로 마시지 않았다. 소금빵을 좋아하며 몇 주 전에 먹었고, 다음에는 학교 근처 브레덴코에 가볼 계획이다.
"""
                }
            },
            "required": ["summary"]
        }
    }
]

In [4]:
context = """
Summarize the following dialog focusing only on the key information shared by the user. Include the provided date in the summary.
[Date]
{date}
[Dialog]
{dialog}
"""

functions = [
    {
        "name": "summarize_conversation",
        "description": "Creates a focused summary of the user's shared information from the conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": """
[Task]
Create a concise summary in Korean focusing only on the key information shared by the user.

[Rules]
1. Include the date provided above in each paragraph of the summary (e.g., "사용자는 2025년 1월 11일에 ~").
2. 육하원칙: '누가, 언제, 어디서, 무엇을, 어떻게, 왜'를 반드시 포함하여 요약.
3. Group the summary into distinct paragraphs based on topics such as '음식', '계획', '취미', '학업', '운동', etc.
   - Each paragraph must start with the topic (e.g., "1. 음식").
   - Insert **exactly two newlines (`\n\n`)** between paragraphs.
   - Inside each paragraph, use single newlines (`\n`) where necessary, such as for listing details or breaking lines naturally.
4. If the user mentions plans for the next day, specify the exact date (e.g., "2025년 1월 14일").
5. Do not use relative terms like "같은 날" or "다음 날"; always specify the exact date.
6. Do not add any extra information or assumptions. If certain details (e.g., time or location) are missing, leave them blank.

[Example Format]
1. 운동
- 2023년 12월 20일 저녁, 집 근처 공원에서 1시간 동안 조깅을 했다. 이유는 건강 관리를 위해서였다.

\n\n

2. 식사
- 2023년 12월 20일 점심에 친구들과 함께 학교 근처 브레덴코에서 소금빵과 크림 모카번을 먹었다. 딸기가 들어간 빵을 좋아한다고 말했다.
- 커피는 잠을 잘 못 자는 문제가 있어서 마시지 않았다.

\n\n

3. 학업
- 2023년 12월 22일부터 기말고사가 시작되며, 준비가 아직 부족하다고 느낀다고 말했다.
"""
                }
            },
            "required": ["summary"]
        }
    }
]

In [5]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from datetime import datetime

def format_date(date_str):
    """
    Converts 'YYMMDD' format into 'YYYY년 MM월 DD일'.
    Args:
        date_str (str): Date string in 'YYMMDD' format.
    Returns:
        str: Formatted date in 'YYYY년 MM월 DD일' format.
    """
    date = datetime.strptime(date_str, "%y%m%d")
    return date.strftime("%Y년 %m월 %d일")


load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")


# OpenAI 클라이언트 초기화
client = OpenAI(api_key=OPENAI_KEY)

# date_list.txt 파일 읽기
date_list_file = "date_list.txt"

# summary 폴더 생성 (없으면 생성)
summary_folder = "summary"
os.makedirs(summary_folder, exist_ok=True)

try:
    with open(date_list_file, "r", encoding="utf-8") as date_file:
        dates = [line.strip() for line in date_file if line.strip()]  # 빈 줄 제거

    for date in dates:
        file_path = f"conversation/conversation_{date}.json"  # 입력 파일 경로
        output_file = f"{summary_folder}/summary_{date}.json"  # 출력 파일 경로

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                conversation = json.load(f)

            # 대화 내용 추출
            dialog = "\n".join([f"{entry['role']}: {entry['content']}" for entry in conversation])

            formatted_date = format_date(date)  # e.g., "2025년 01월 13일"

            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": context.format(dialog=dialog, date=formatted_date)}
                ],
                tools=[{
                    "type": "function",
                    "function": functions[0]
                }],
                tool_choice={"type": "function", "function": {"name": "summarize_conversation"}}
            )

            # 요약 출력
            function_call = response.choices[0].message.tool_calls[0].function
            function_args = json.loads(function_call.arguments)

            summary = function_args['summary']
            print(f"날짜 {formatted_date}의 대화 요약:")
            print(f"요약: {summary}")

            # 문단을 "\n\n" 기준으로 나누기
            paragraphs = summary.split("\n\n")  # 두 번의 줄바꿈으로 문단 구분
            cleaned_paragraphs = [para.strip() for para in paragraphs if para.strip()]  # 빈 문단 제거

            # JSON으로 저장
            output_data = {
                "date": formatted_date,
                "summary_paragraphs": cleaned_paragraphs  # 문단별 저장
            }

            with open(output_file, "w", encoding="utf-8") as output_f:
                json.dump(output_data, output_f, ensure_ascii=False, indent=4)

            print(f"날짜 {formatted_date}의 요약이 '{output_file}' 파일에 저장되었습니다.\n")

        except FileNotFoundError:
            print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        except json.JSONDecodeError:
            print(f"오류: '{file_path}' 파일의 JSON 형식이 올바르지 않습니다.")
        except Exception as e:
            print(f"날짜 {date} 처리 중 오류 발생: {str(e)}\n")


except FileNotFoundError:
    print("오류: 'date_list.txt' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {str(e)}")

날짜 2025년 01월 11일의 대화 요약:
요약: 1. 음식
- 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다. 그 빵에는 크림과 딸기 모카번이 들어있다고 말했다.

2. 계획
- 2025년 1월 12일에는 다시 마라탕을 먹을 계획이라고 언급했다.
날짜 2025년 01월 11일의 요약이 'summary/summary_250111.json' 파일에 저장되었습니다.

날짜 2025년 01월 12일의 대화 요약:
요약: 1. 음식
- 사용자는 2025년 1월 12일에 매우 기분이 좋았고, 그냥 좋은 기분이어서 특별한 이유는 없다고 했다.
- 사용자는 맛있는 음식을 먹는 것을 좋아한다. 그날 딸기가 들어간 크림 모카번을 먹었는데, 커피 대신 빵과 함께 먹었다.
- 사용자는 소금빵도 좋아해하며, 마지막으로 먹은 것은 몇 주 전이었다. 그리고 학교 근처의 브레덴코에서 다음에 또 먹을 계획이 있다.

2. 취향
- 사용자는 크림과 과일이 들어간 빵을 좋아하며, 치즈 빵은 별로 좋아하지 않는다. 과일 디저트 중에서는 케이크를 제일 좋아한다.

3. 최근의 행동
- 사용자는 최근에 케이크를 먹은 것은 예수님의 생일 때였으며, 그 때 먹은 케이크는 생크림 케이크였다.
날짜 2025년 01월 12일의 요약이 'summary/summary_250112.json' 파일에 저장되었습니다.

날짜 2025년 01월 13일의 대화 요약:
요약: 1. 편안한 대화
- 2025년 1월 13일, 사용자는 가장 즐거웠던 순간이 AI와 이야기하는 현재 순간이라고 말하였습니다.

2. 음식
- 사용자는 2025년 1월 13일 점심에 '선재 업고 튀어라'라는 드라마를 보면서 엄마가 싸준 마라탕 도시락을 먹을 계획이라고 밝혔습니다. 사용자는 매운 음식을 아주 좋아하며, 특히 떡볶이와 짬뽕이 떠오르는 매운 음식이라고 언급하였습니다. 또한, 주말에도 마라탕을 먹을 계획이라고 말하